# How to calculate risk-adjusted with the Sortino Ration

The Sortino ratio is calculated in the same manner as the Sharpe ratio, except it focuses on downside risk rather than volatility in general. It is calculated by dividing an asset's excess returns (realized returns minus the risk-free rate) and dividing by the standard deviation of the downside.

$$ Sortino_Ratio = \frac{r_{p}-r_{f}}{\sigma_{d}} $$

 - Portfolio Return (rp): The return on a portfolio, either on a historical basis (i.e. actual results).
 - Risk-Free Rate (rf)  : The risk-free rate is the return received on default-free securities.
 - Downside Standard Deviation (σd): The standard deviation of solely the investment’s or portfolio’s negative returns, i.e. the downside deviation.

Sources:
- https://www.investopedia.com/ask/answers/010815/what-difference-between-sharpe-ratio-and-sortino-ratio.asp#:~:text=The%20Sortino%20ratio%20is%20calculated,standard%20deviation%20of%20the%20downside.
- https://www.pyquantnews.com/the-pyquant-newsletter/how-to-measure-your-risk-adjusted-returns-sortino 


In [1]:
pip install yfinance

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import yfinance as yf
import pandas as pd

In [6]:
data = yf.download(["SPY","AAPL"], start="2020-01-01", end="2025-03-31")

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  2 of 2 completed


In [7]:
data

Price            Close                    High                     Low  \
Ticker            AAPL         SPY        AAPL         SPY        AAPL   
Date                                                                     
2020-01-02   72.716057  300.291534   72.776583  300.310039   71.466797   
2020-01-03   72.009117  298.017700   72.771745  299.154653   71.783962   
2020-01-06   72.582886  299.154633   72.621623  299.237820   70.876053   
2020-01-07   72.241539  298.313477   72.849216  299.062193   72.021223   
2020-01-08   73.403641  299.903381   73.706271  301.132746   71.943751   
...                ...         ...         ...         ...         ...   
2025-03-24  220.729996  574.080017  221.479996  575.150024  218.580002   
2025-03-25  223.750000  575.460022  224.100006  576.409973  220.080002   
2025-03-26  221.529999  568.590027  225.020004  576.330017  220.470001   
2025-03-27  223.850006  567.080017  224.990005  570.900024  220.559998   
2025-03-28  217.899994  555.659973  223.809998  566.270020  217.679993   

Price                         Open                 Volume            
Ticker             SPY        AAPL         SPY       AAPL       SPY  
Date                                                                 
2020-01-02  298.128574   71.721004  299.062170  135480400  59151200  
2020-01-03  296.806812   71.941328  296.862270  146322800  77709700  
2020-01-06  296.122758   71.127844  296.242927  118387200  55653900  
2020-01-07  297.860529   72.592586  298.581516  108872000  40496400  
2020-01-08  298.258050   71.943751  298.507613  132079200  68296000  
...                ...         ...         ...        ...       ...  
2025-03-24  570.200012  221.000000  570.799988   44299500  58766800  
2025-03-25  573.690002  220.770004  575.299988   34493600  38355700  
2025-03-26  567.190002  223.509995  575.190002   34466100  51848300  
2025-03-27  564.940002  221.389999  567.179993   37094800  42164200  
2025-03-28  555.070007  221.669998  565.530029   39818600  71662700  

[1317 rows x 10 columns]